<a href="https://colab.research.google.com/github/mateuszklinowski/TensorflowCourse/blob/master/Comma_ai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install tf-nightly-gpu
!pip install "tensorflow_hub==0.4.0"
!pip install -U tensorflow_datasets


In [4]:
from __future__ import absolute_import, division, print_function, unicode_literals
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds
import cv2.cv2 as cv
import cv2 as cv2
import time
from tensorflow.keras import layers
import logging

logger = tf.get_logger()
logger.setLevel(logging.ERROR)

tf.enable_eager_execution()


  TensorFlow's `tf-nightly` package will soon be updated to TensorFlow 2.0.

  Please upgrade your code to TensorFlow 2.0:
    * https://www.tensorflow.org/beta/guide/migration_guide

  Or install the latest stable TensorFlow 1.X release:
    * `pip install -U "tensorflow==1.*"`

  Otherwise your code may be broken by the change.

  


In [5]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [0]:
!ls drive/My\ Drive/speedchallenge/combined

In [0]:
import pathlib
data_root = pathlib.Path('/content/drive/My Drive/speedchallenge/combined')
AUTOTUNE = tf.data.experimental.AUTOTUNE

Create list containing all images paths

In [0]:
import random
all_image_paths = list()
for item in data_root.iterdir():
    all_image_paths.append(item)

all_image_paths = [str(path) for path in all_image_paths]
random.shuffle(all_image_paths)

Print images length:


In [9]:
image_count = len(all_image_paths)
print("N: ", image_count)

N:  20400


In [0]:
for item in data_root.iterdir():
    path = item.as_posix()
    if '(' in path:
      print(path)

Create utils functions for processing images


In [0]:
import tensorflow as tf

def preprocess_image(image):
    image = tf.image.decode_png(image, channels=3)
    image = tf.image.resize(image, [224, 224])
    image /= 255.0
    return image

def load_and_process_image(image_path):
    image = tf.read_file(image_path)
    return preprocess_image(image)

Create TF Dataset's from images paths

In [0]:
path_ds = tf.data.Dataset.from_tensor_slices(all_image_paths)
image_ds = path_ds.map(load_and_process_image, num_parallel_calls=AUTOTUNE)

In [13]:
predictor = open('/content/drive/My Drive/speedchallenge/train.txt', 'r').readlines()
predictor = list(map(lambda x: float(x), predictor))
values_ds = tf.data.Dataset.from_tensor_slices(tf.cast(predictor, tf.double))

for label in values_ds.take(10):
  print(label)

tf.Tensor(28.10556983947754, shape=(), dtype=float64)
tf.Tensor(28.10556983947754, shape=(), dtype=float64)
tf.Tensor(28.10652732849121, shape=(), dtype=float64)
tf.Tensor(28.130403518676758, shape=(), dtype=float64)
tf.Tensor(28.109243392944336, shape=(), dtype=float64)
tf.Tensor(28.088571548461914, shape=(), dtype=float64)
tf.Tensor(28.034210205078125, shape=(), dtype=float64)
tf.Tensor(28.018491744995117, shape=(), dtype=float64)
tf.Tensor(27.986623764038086, shape=(), dtype=float64)
tf.Tensor(28.0163516998291, shape=(), dtype=float64)


Combining images and values into one DataSet

In [14]:
image_label_ds = tf.data.Dataset.zip((image_ds, values_ds))
print("DataSet: ", image_label_ds)


DataSet:  <DatasetV1Adapter shapes: ((224, 224, 3), ()), types: (tf.float32, tf.float64)>


In [0]:
# Reformat Images and Create Batches
BATCH_SIZE = 32
DATASET_SIZE = 20400

In [0]:
image_label_ds = image_label_ds.shuffle(3670)

train_ds = image_label_ds.take(int(0.8*20400))
test_ds = image_label_ds.skip(int(0.8*20400))

train_batches = train_ds.shuffle(3670).batch(BATCH_SIZE)
test_batches = test_ds.shuffle(3670).batch(BATCH_SIZE)


 Do Simple Transfer Learning with TensorFlow Hub

In [0]:
URL = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"
feature_extractor = hub.KerasLayer(URL, input_shape=(224, 224, 3), trainable=False)
feature_extractor.trainable = False

model = tf.keras.Sequential([
    feature_extractor,
    tf.keras.layers.Dense(units=1, activation=tf.nn.sigmoid)
])


Model summary

In [18]:

print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 1280)              2257984   
_________________________________________________________________
dense (Dense)                (None, 1)                 1281      
Total params: 2,259,265
Trainable params: 1,281
Non-trainable params: 2,257,984
_________________________________________________________________
None


In [0]:
model.compile(
  optimizer='adam',
  loss='mean_squared_error',
  metrics=['mean_squared_error']
)

In [0]:
EPOCHS = 2

In [0]:
history = model.fit(train_batches,
                    epochs=EPOCHS,
                    validation_data=test_batches,
                    verbose=1)

Epoch 1/2


In [0]:
plt.xlabel('Epoch Number')
plt.ylabel("Loss Magnitude")
plt.plot(history.history['loss'])